## Part 2: Analyzing Student Profiles

### Assignment Overview
Analyze the Student Profiles of a business training institute. Use plotly to plot charts and provide compelling insights.

#### Objective
Analyze the relationship between student nationality amd their highest qualification, age, course name/code and course funding

**Task:**
- Two charts using plotly express or graph objects
- One plotly dash that may contain one or more charts and include interactive
elements such as dropdown menu.


# Data Cleaning

In [281]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Loading the data

In [282]:
# Load the dataset
file_path = './Data/Student Profiles.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataset to understand its structure
display(data.head())

,STUDENT ID,SALUTATION,GENDER,NATIONALITY,LOCAL,FOREIGN,DOB,HIGHEST QUALIFICATION,NAME OF QUALIFICATION AND INSTITUTION,DATE ATTAINED HIGHEST QUALIFICATION,DESIGNATION,INTAKE NO,COMMENCEMENT DATE,COMPLETION DATE,FULL-TIME OR PART-TIME,COURSE FUNDING,REGISTRATION FEE,PAYMENT MODE,COURSE FEE,GPA
0,2020/1101-013/001,Ms,F,Singaporean,Y,,04/03/1978,Certificate,"Certificate in Office Skills, ITE",2016-11-06,"Snr Associate, Client Services",13th,2020-01-06,2021-04-06 00:00:00,Part-Time,Individual - SFC,107,NETS,1712,3.0
1,2020/1101-013/002,Ms,F,Singaporean,Y,,23/03/1966,Certificate,"WSQ Higher Certificate in Human Resources, WPL...",2018-02-06,Admin Officer,13th,2020-01-06,2021-04-06 00:00:00,Part-Time,Individual,107,NETS,1712,2.5
2,2020/1101-013/003,Ms,F,Singaporean,Y,NaN,04/12/1988,Certificate,"Nitec in Service Skills (Office), ITE",2016-11-06,Admin Assistant,13th,2020-01-06,2021-04-06 00:00:00,Part-Time,Sponsored - no SDF,107,CC JPM,1712,2.0
3,2020/1101-014/004,Ms,F,Singaporean,Y,NaN,27/11/1991,Degree,Bachelor Science (Facilities & Events Manageme...,2017-06-10,HR Administrator,14th,2020-03-10,2021-06-09 00:00:00,Part-Time,Individual - SFC,107,NETS,1212,2.5
4,2020/1101-014/005,Ms,F,Singaporean,Y,,23/04/1985,Diploma,"Diploma in Procurement and Supply Management, ...",2016-08-10,Purchasing Executive,14th,2020-03-10,2021-06-09 00:00:00,Part-Time,Individual - SFC,107,NETS,1712,2.4


## Check For Any Duplicated STUDENT ID Values

In [283]:
# Check for duplicated values in the 'STUDENT ID' column
duplicated_values = data[data.duplicated(subset='STUDENT ID')]

# Print the duplicated values
print(duplicated_values)

# Print the number of duplicated values
print(f"Number of duplicated STUDENT IDs: {duplicated_values.shape[0]}")


Empty DataFrame
Columns: [STUDENT ID, SALUTATION, GENDER, NATIONALITY, LOCAL, FOREIGN, DOB, HIGHEST QUALIFICATION, NAME OF QUALIFICATION AND INSTITUTION, DATE ATTAINED HIGHEST QUALIFICATION, DESIGNATION, INTAKE NO, COMMENCEMENT DATE, COMPLETION DATE, FULL-TIME OR PART-TIME, COURSE FUNDING, REGISTRATION FEE, PAYMENT MODE, COURSE FEE, GPA]
Index: []
Number of duplicated STUDENT IDs: 0


#### **Output Details:**
All the Student IDs are unique, indicating that no changes are needed.

## Data Cleaning for Salutation and Gender

After checking, both Salutation and Gender columns appear to be correctly input into the dataset, requiring no cleaning.


## Clean the Nationality Column and Display the Nationality Column

In [284]:
# Clean the 'NATIONALITY' column
def clean_nationality(nationality):
    # Remove leading/trailing whitespace
    nationality = nationality.strip()
    
    # Standardize the suffixes
    nationality = nationality.replace(" (PR)", "(PR)")
    
    # Handle inconsistencies
    nationality = nationality.replace("Indian ", "Indian")
    nationality = nationality.replace("Chinese ", "Chinese")
    nationality = nationality.replace("Malaysian ", "Malaysian")
    
    return nationality

# Apply the cleaning function to the 'NATIONALITY' column
data['NATIONALITY'] = data['NATIONALITY'].apply(clean_nationality)

# Check the cleaned 'NATIONALITY' column
unique_nationalities = data['NATIONALITY'].unique()
print("Unique Nationalities:", unique_nationalities)
print("Number of Unique Nationalities:", len(unique_nationalities))


Unique Nationalities: ['Singaporean' 'Malaysian(PR)' 'Chinese' 'Malaysian' 'Indian'
 'Netherlands' 'Indian(PR)' 'Chinese(PR)' 'Filipino' 'Filipino(PR)'
 'Vietnamese' 'Myanmar' 'China(PR)' 'Chinese\n(China)' 'Indonesian(PR)']
Number of Unique Nationalities: 15


#### Changes Made

- `'Chinese\n(China)'` is replaced with `'Chinese'`.
- `'China(PR)'` is replaced with `'Chinese(PR)'`.


In [285]:
# Combine variations in the 'NATIONALITY' column
data['NATIONALITY'] = data['NATIONALITY'].replace({
    'Chinese\n(China)': 'Chinese',
    'China(PR)': 'Chinese(PR)'
})

# Check the cleaned 'NATIONALITY' column again
unique_nationalities = data['NATIONALITY'].unique()
print("Unique Nationalities:", unique_nationalities)
print("Number of Unique Nationalities:", len(unique_nationalities))


Unique Nationalities: ['Singaporean' 'Malaysian(PR)' 'Chinese' 'Malaysian' 'Indian'
 'Netherlands' 'Indian(PR)' 'Chinese(PR)' 'Filipino' 'Filipino(PR)'
 'Vietnamese' 'Myanmar' 'Indonesian(PR)']
Number of Unique Nationalities: 13


## 'Local' and 'Foregin Column Data Cleaning

In [286]:
# Replace 'NaN' with empty string ('') in the LOCAL and FOREIGN columns
data['LOCAL'].fillna('', inplace=True)
data['FOREIGN'].fillna('', inplace=True)

In [287]:
# Iterate through each row in the DataFrame
for index, row in data.iterrows():
    nationality = row['NATIONALITY']
    
    # Modify LOCAL and FOREIGN columns based on NATIONALITY
    if nationality == 'Singaporean':
        data.at[index, 'LOCAL'] = 'Y'
        data.at[index, 'FOREIGN'] = 'N'
    else:
        data.at[index, 'LOCAL'] = 'N'
        data.at[index, 'FOREIGN'] = 'Y'

### Data Transformation

1. Replaced 'NaN' with empty strings ('') in the LOCAL and FOREIGN columns.
2. Modified LOCAL and FOREIGN columns based on NATIONALITY.

## Cleaning the 'DOB' Column

In [288]:
import pandas as pd
import re
from datetime import datetime

In [289]:
# Define a function to check the date format
def is_valid_date(date):
    pattern = re.compile(r'\b\d{2}/\d{2}/\d{4}\b')
    return bool(pattern.match(date))

# Function to correct the date format
def correct_date_format(date):
    try:
        return datetime.strptime(date, '%d-%b-%Y').strftime('%d/%m/%Y')
    except ValueError:
        try:
            return datetime.strptime(date, '%d-%m-%Y').strftime('%d/%m/%Y')
        except ValueError:
            return date

# Apply the function to the 'DOB' column to check validity
data['DOB_valid'] = data['DOB'].apply(is_valid_date)

# Print rows where DOB format is invalid
invalid_dob = data[~data['DOB_valid']]
print("Invalid DOB entries:")
print(invalid_dob[['DOB']])

# Count of invalid date formats
invalid_count = len(invalid_dob)
print(f"Number of invalid DOB entries: {invalid_count}")

# Correct the date format for invalid entries
data.loc[~data['DOB_valid'], 'DOB'] = data.loc[~data['DOB_valid'], 'DOB'].apply(correct_date_format)

# Recheck the 'DOB' column to check validity again after corrections
data['DOB_valid'] = data['DOB'].apply(is_valid_date)

# Print rows where DOB format is still invalid after correction
still_invalid_dob = data[~data['DOB_valid']]
print("Still invalid DOB entries after correction:")
print(still_invalid_dob[['DOB']])

# Count of still invalid date formats
still_invalid_count = len(still_invalid_dob)
print(f"Number of still invalid DOB entries after correction: {still_invalid_count}")

# Remove the helper column if not needed
data.drop(columns=['DOB_valid'], inplace=True)

Invalid DOB entries:
            DOB
59  13-Feb-1984
60  13-Jul-1987
61  15-Jul-1994
94   16-07-1991
Number of invalid DOB entries: 4
Still invalid DOB entries after correction:
Empty DataFrame
Columns: [DOB]
Index: []
Number of still invalid DOB entries after correction: 0


#### Output:

#### 4 invalid formatted DOB dates were identified and correctly modified.

## Cleaning HIGHEST QUALIFICATION column

In [290]:
# Print unique values from the 'HIGHEST QUALIFICATION' column
unique_values = data['HIGHEST QUALIFICATION'].unique()
print(unique_values)


['Certificate' 'Degree' 'Diploma' 'Master' 'Graduate Diploma' ' '
 'Postgraduate Diploma' 'Postgraduate Diploma/\nDegree']


In [291]:
import numpy as np

# Clean and replace empty strings in 'HIGHEST QUALIFICATION' column
data['HIGHEST QUALIFICATION'] = data['HIGHEST QUALIFICATION'].str.strip().str.replace('\n', '')
data['HIGHEST QUALIFICATION'] = data['HIGHEST QUALIFICATION'].replace('', 'Unknown')

# Print unique values after cleaning and replacement
unique_values = data['HIGHEST QUALIFICATION'].unique()
print(unique_values)

['Certificate' 'Degree' 'Diploma' 'Master' 'Graduate Diploma' 'Unknown'
 'Postgraduate Diploma' 'Postgraduate Diploma/Degree']


#### Combine Graduate Diploma, Postgraduate Diploma and Postgraduate Diploma/Degree into A New Category called 'Graduate Diploma' Category

In [292]:
import numpy as np
import pandas as pd

# Assuming 'data' is your DataFrame containing the 'HIGHEST QUALIFICATION' column

# Clean and replace empty strings in 'HIGHEST QUALIFICATION' column
data['HIGHEST QUALIFICATION'] = data['HIGHEST QUALIFICATION'].str.strip().replace('', 'Unknown')

# Create a mapping dictionary for replacing values
qualification_mapping = {
    'Graduate Diploma': ['Graduate Diploma', 'Postgraduate Diploma', 'Postgraduate Diploma/Degree']
}

# Replace values using the mapping dictionary
for key, value_list in qualification_mapping.items():
    data['HIGHEST QUALIFICATION'].replace(value_list, key, inplace=True)

# Print unique values after cleaning and replacement
unique_values = data['HIGHEST QUALIFICATION'].unique()
print(unique_values)


['Certificate' 'Degree' 'Diploma' 'Master' 'Graduate Diploma' 'Unknown']


In [293]:
# Verify the changes
print(data['HIGHEST QUALIFICATION'].value_counts())

HIGHEST QUALIFICATION
Degree              104
Certificate          69
Diploma              56
Master               11
Graduate Diploma      3
Unknown               1
Name: count, dtype: int64


## Checking 'COMMENCEMENT DATE' and 'COMPLETION DATE' columns

In [294]:
# Check for missing values
missing_values = data[['COMMENCEMENT DATE', 'COMPLETION DATE']].isnull().sum()
print("Missing Values:")
print(missing_values)

# Convert to datetime format and suppress errors
data['COMMENCEMENT DATE'] = pd.to_datetime(data['COMMENCEMENT DATE'], format='%d/%m/%Y', errors='coerce')
data['COMPLETION DATE'] = pd.to_datetime(data['COMPLETION DATE'], format='%d/%m/%Y', errors='coerce')

# Check if there were any conversion errors
if data['COMMENCEMENT DATE'].isnull().any() or data['COMPLETION DATE'].isnull().any():
    print("Error in date conversion. Please check the date formats.")
else:
    print("No wrongly formatted dates.")

Missing Values:
COMMENCEMENT DATE    0
COMPLETION DATE      0
dtype: int64
No wrongly formatted dates.


## Create a new Column called Age(at course completion)

In [295]:
data['DOB'] = pd.to_datetime(data['DOB'], format='%d/%m/%Y')
data['COMPLETION DATE'] = pd.to_datetime(data['COMPLETION DATE'], format='%d/%m/%Y')

# Assuming 'COMPLETION DATE' and 'DOB' are datetime columns
data['Age(at course completion)'] = (data['COMPLETION DATE'] - data['DOB']).dt.days // 365

## Check column 'COURSE FUNDING', clean data and permute

In [296]:
# Assuming 'data' is your DataFrame containing the 'COURSE FUNDING' column
unique_values = data['COURSE FUNDING'].unique()

print("Unique values in 'COURSE FUNDING':")
for value in unique_values:
    print(value)

Unique values in 'COURSE FUNDING':
Individual - SFC
Individual
Sponsored - no SDF
Sponsored
Individual   
Individual 
Individual-SFC
Sponsored 
Individual - waived App Fee
Individual - SFC + $1000 SCHOLARSHIP
Indivodual
Sponsored   
Individual  
Sponsored  
Sponsored-no SDF
Indvidual - SFC
Sponsored - SDF


In [297]:
# Define a dictionary for replacements
replacements = {
    'Individual': 'Individual',
    'Individual - waived App Fee': 'Individual',
    'Indivodual': 'Individual',
    'Individual - SFC': 'Individual - SFC',
    'Individual-SFC': 'Individual - SFC',
    'Individual - SFC + $1000 SCHOLARSHIP': 'Individual - SFC',
    'Indvidual - SFC': 'Individual - SFC',
    'Sponsored - no SDF': 'Sponsored',
    'Sponsored': 'Sponsored',
    'Sponsored-no SDF': 'Sponsored'
}

# Apply the replacements to the 'COURSE FUNDING' column
data['COURSE FUNDING'] = data['COURSE FUNDING'].replace(replacements)

# Display the unique values after replacement to verify the changes
unique_values_after_replacement = data['COURSE FUNDING'].unique()

print("Unique values in 'COURSE FUNDING' after replacement:")
for value in unique_values_after_replacement:
    print(value)

Unique values in 'COURSE FUNDING' after replacement:
Individual - SFC
Individual
Sponsored
Individual   
Individual 
Sponsored 
Sponsored   
Individual  
Sponsored  
Sponsored - SDF


In [298]:
# Define a function to clean and standardize the 'COURSE FUNDING' values
def clean_course_funding(value):
    value = value.strip().lower()  # Remove leading/trailing spaces and convert to lowercase
    if value in ['individual', 'individual - waived app fee', 'indivodual']:
        return 'Individual'
    elif value in ['individual - sfc', 'individual-sfc', 'individual - sfc + $1000 scholarship', 'indvidual - sfc']:
        return 'Individual - SFC'
    elif value in ['sponsored', 'sponsored - no sdf', 'sponsored-no sdf']:
        return 'Sponsored'
    else:
        return value.title()  # Capitalize for consistency

# Apply the cleaning function to the 'COURSE FUNDING' column
data['COURSE FUNDING'] = data['COURSE FUNDING'].apply(clean_course_funding)

# Display the unique values after replacement to verify the changes
unique_values_after_replacement = data['COURSE FUNDING'].unique()

print("Unique values in 'COURSE FUNDING' after replacement:")
for value in unique_values_after_replacement:
    print(value)

Unique values in 'COURSE FUNDING' after replacement:
Individual - SFC
Individual
Sponsored
Sponsored - Sdf


## Create a new column called COURSE NAME, AND COURSE CODE

In [299]:
df_courseCodes = pd.read_excel("./Data/Course Codes.xlsx")

In [300]:
# Example df_courseCodes setup (replace with your actual df_courseCodes loading)
df_courseCodes = pd.DataFrame({
    'CourseCode': ['1101', '1102', '2101', '2102', '2013', '5112', '5113'],
    'CourseName': [
        'Diploma in Business Administration',
        'Diploma in Business Analytics',
        'Certificate in HR Management',
        'Certificate in Digital Marketing',
        'Certificate in Tourism Management',
        'Bachelor of Business Administration',
        'Master of Business Administration'
    ]
})

# Extract course code from 'STUDENT ID' column
data['CourseCode'] = data['STUDENT ID'].str.extract(r'/(\d+)-')

# Merge data with df_courseCodes based on 'CourseCode' directly into 'data'
data = pd.merge(data, df_courseCodes, on='CourseCode', how='left')

# # Check the updated data dataframe with course names
# print(data)

In [301]:
# # Specify the file path where you want to save the Excel file
# excel_file_path = r'C:\Users\lhchu\OneDrive\Desktop\DAAA (SP)2A03\DAVI\CA2\Data\data_export.xlsx'

# # Export the DataFrame to Excel
# data.to_excel(excel_file_path, index=False)

# print(f"DataFrame successfully exported to Excel file: {excel_file_path}")

In [302]:
# from IPython.display import display

# # Assuming 'data' is your DataFrame
# # Set display options to show all rows
# pd.set_option('display.max_rows', None)  # Show all rows

# # Display the DataFrame
# display(data)

## Plotly Data Analysis

#### Sunburst chart to see relationship between Nationaility and (Highest Qualification and Age)

In [90]:
import plotly.express as px
import pandas as pd

# Load data
data = pd.read_excel('./Data/data_export.xlsx')

# Strip any leading or trailing whitespace characters from column names
data.columns = data.columns.str.strip()

# Group by Nationality and Highest Qualification and count occurrences
grouped_data = data.groupby(['NATIONALITY', 'HIGHEST QUALIFICATION']).size().reset_index(name='Count')

# Calculate total counts for percentages
total_count = grouped_data['Count'].sum()

# Calculate percentages
grouped_data['Percentage'] = (grouped_data['Count'] / total_count) * 100

# Create the Sunburst chart with percentages in hover data
fig = px.sunburst(
    grouped_data,
    path=['NATIONALITY', 'HIGHEST QUALIFICATION'],
    values='Count',
    title='Sunburst Chart: Highest Qualification by Nationality',
    labels={'NATIONALITY': 'Nationality', 'HIGHEST QUALIFICATION': 'Highest Qualification', 'Count': 'Count'},
    hover_data={'Percentage': True}  # Include percentage in hover data
)

# Update the hover template to include percentages
fig.update_traces(
    hovertemplate='%{label}<br>Count: %{value}<br>Percentage: %{customdata:.2f}%<extra></extra>'
)

# Update layout
fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=40, r=40, t=40, b=40),
    font=dict(size=14)
)

# Show the figure
fig.show()


### Insights for Sunburst Chart: Highest Qualification by Nationality

- **Distribution of Qualifications**: The chart shows that certain nationalities, such as those in the "Singaporean" and "Malaysian" categories, have a higher representation in advanced qualifications like Bachelor's and Master's degrees. This indicates a trend of higher education levels within these nationalities.
- **Proportional Representation**: Hovering over the segments, it is evident that a significant portion of the "Singaporean" population holds at least a Bachelor's degree, with a noticeable percentage also holding Master's degrees. For the "Malaysian" group, there is a similar trend with a high percentage in Bachelor's degrees and some representation in higher qualifications.
- **Predominant Qualifications**: The largest segments in the sunburst chart highlight that Bachelor's degrees are the most common highest qualifications among both "Singaporean" and "Malaysian" nationalities. This shows that Bachelor's degrees are the prevalent educational achievement within the dataset for these groups.


## Grouped Bar Chart on Local/Foreign Students vs their Funding type

In [88]:
import plotly.express as px
import pandas as pd

# Load data
data = pd.read_excel('./Data/data_export.xlsx')

# Strip any leading or trailing whitespace characters from column names
data.columns = data.columns.str.strip()

# Create a new column for classification
def classify_nationality(row):
    if row['LOCAL'] == 'Y':
        return 'LOCAL'
    elif 'PR' in str(row['NATIONALITY']).upper():  # Check for PR in the nationality or residency status
        return 'FOREIGN (PR)'
    elif row['FOREIGN'] == 'Y':
        return 'FOREIGN'
    else:
        return 'Unknown'

data['Nationality_Group'] = data.apply(classify_nationality, axis=1)

# Group by Nationality and Course Funding and count occurrences
grouped_data = data.groupby(['Nationality_Group', 'COURSE FUNDING']).size().reset_index(name='Count')

# Create the grouped bar chart
fig = px.bar(
    grouped_data,
    x='Nationality_Group',
    y='Count',
    color='COURSE FUNDING',
    barmode='group',
    title='Grouped Bar Chart: Course Funding by NATIONALITY GROUP',
    labels={'Nationality_Group': 'Nationality Group', 'Count': 'Count', 'COURSE FUNDING': 'Course Funding'}
)

# Update layout
fig.update_layout(
    width=1000,
    height=600,
    font=dict(size=14)
)

# Show the figure
fig.show()



#### Foreign Students
- The majority of foreign students are self-funded (Individual), with approximately 20 students.
- A smaller number of foreign students are sponsored, with about 5 students.

#### Foreign (PR) Students
- Similar to foreign students, a significant portion of foreign (PR) students are self-funded (Individual), with around 15 students.
- There are minimal instances of other funding types among foreign (PR) students, with less than 5 students for each category.

#### Local Students
- Local students exhibit the highest number of self-funded (Individual) students, with a count close to 90.
- A substantial number of local students also receive Individual - SFC funding, with about 70 students.
- There are around 40 local students who are sponsored, and a few are under the Sponsored - Sdf category.

### Summary
- **Self-Funded Dominance**: Across all nationality groups, self-funding (Individual) is the most common form of course funding.
- **Local vs. Foreign**: Local students have a significantly higher count in both self-funding (Individual) and Individual - SFC categories compared to foreign and foreign (PR) students.
- **Diverse Funding**: Local students show a more diverse distribution of funding types, including a notable number of sponsored students and those under Individual - SFC.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load data
data = pd.read_excel('./Data/data_export.xlsx')

# Strip any leading or trailing whitespace characters from column names
data.columns = data.columns.str.strip()

# Create a new column for classification
def classify_nationality(row):
    if row['LOCAL'] == 'Y':
        return 'LOCAL'
    elif 'PR' in str(row['NATIONALITY']).upper():  # Check for PR in the nationality or residency status
        return 'FOREIGN (PR)'
    elif row['FOREIGN'] == 'Y':
        return 'FOREIGN'
    else:
        return 'Unknown'

data['Nationality_Group'] = data.apply(classify_nationality, axis=1)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1('Interactive Dashboard'),

    # Dropdown menu for selecting nationality group
    dcc.Dropdown(
        id='nationality-group-dropdown',
        options=[
            {'label': 'LOCAL', 'value': 'LOCAL'},
            {'label': 'FOREIGN', 'value': 'FOREIGN'},
            {'label': 'FOREIGN (PR)', 'value': 'FOREIGN (PR)'}
        ],
        value='LOCAL'  # Default value
    ),

    # Container for the histogram
    html.Div(id='histogram-container'),

    # Container for the donut chart
    html.Div(id='donut-chart-container')
])

# Callback to update histogram based on dropdown selection
@app.callback(
    Output('histogram-container', 'children'),
    [Input('nationality-group-dropdown', 'value')]
)
def update_histogram(selected_group):
    # Filter data based on selected nationality group
    filtered_data = data[data['Nationality_Group'] == selected_group]

    # Create histogram for age
    fig = px.histogram(
        filtered_data,
        x='Age(at course completion)',
        title=f'Histogram of Ages for {selected_group}',
        labels={'Age(at course completion)': 'Age at Course Completion'}
    )

    # Update layout for better readability and size
    fig.update_layout(
        width=1000,
        height=600,
        font=dict(size=14)
    )

    return dcc.Graph(figure=fig)

# Callback to update donut chart based on dropdown selection
@app.callback(
    Output('donut-chart-container', 'children'),
    [Input('nationality-group-dropdown', 'value')]
)
def update_donut_chart(selected_group):
    # Filter data based on selected nationality group
    filtered_data = data[data['Nationality_Group'] == selected_group]

    # Create donut chart for course distribution
    fig = px.pie(
        filtered_data,
        names='CourseName',
        title=f'Distribution of Courses for {selected_group}',
        hole=0.4  # Creates the donut effect
    )

    # Update layout for better readability and size
    fig.update_layout(
        width=1000,
        height=600,
        font=dict(size=14)
    )

    return dcc.Graph(figure=fig)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\lhchu\AppData\Local\Temp\ipykernel_23924\2181200977.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\lhchu\AppData\Local\Temp\ipykernel_23924\2181200977.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### LOCAL Group

- **Age Distribution:** Primarily 25-35 years, with significant participation around 25-30 and 35-40 years.
- **Course Preference:** The most popular course is the Certificate in Digital Marketing (38.6%). Other popular courses include the Diploma in Business Administration (20.1%) and various certificates and degrees with smaller percentages.

### FOREIGN Group

- **Age Distribution:** Mainly 25-35 years, peaking at 30-35 years.
- **Course Preference:** A strong preference for the Certificate in Digital Marketing (44.4%) and Diploma in Business Administration (22.2%). Other courses have lower enrollment percentages.

### FOREIGN (PR) Group

- **Age Distribution:** Concentrated in the 25-35 years range, peaking at 30-35 years.
- **Course Preference:** The Certificate in Digital Marketing is the most popular (57.1%), with the Diploma in Business Administration also notable (21.4%). Other courses have minimal enrollment.


### Recommendation
Based on the analysis, it is recommended to tailor specific programs or marketing strategies towards the 25-35 years age range, as this is the most represented age group across all categories. For the LOCAL group, additional focus could be placed on engaging individuals in the 35-40 years range, given the secondary peak observed.